<a href="https://colab.research.google.com/github/ll1195831146/Infor7374-AI/blob/master/Assignment1/Team%207%20-%20Metric.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Metric**

In [2]:
from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import os
from keras import metrics

Using TensorFlow backend.


In [0]:
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_cifar10_trained_model.h5'

In [4]:
# The data, split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

170500096/170498071 [==============================] - 39s 0us/step
x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [0]:
batch_size = 32
num_classes = 10
epochs = 5
data_augmentation = True
num_predictions = 20

In [0]:
# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [7]:
# initiate RMSprop optimizer
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

Instructions for updating:
Colocations handled automatically by placer.


In [0]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

In [9]:
print('Using real-time data augmentation.')
# This will do preprocessing and realtime data augmentation:
datagen = ImageDataGenerator(
    featurewise_center=False,  # set input mean to 0 over the dataset
    samplewise_center=False,  # set each sample mean to 0
    featurewise_std_normalization=False,  # divide inputs by std of the dataset
    samplewise_std_normalization=False,  # divide each input by its std
    zca_whitening=False,  # apply ZCA whitening
    zca_epsilon=1e-06,  # epsilon for ZCA whitening
    rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
    # randomly shift images horizontally (fraction of total width)
    width_shift_range=0.1,
    # randomly shift images vertically (fraction of total height)
    height_shift_range=0.1,
    shear_range=0.,  # set range for random shear
    zoom_range=0.,  # set range for random zoom
    channel_shift_range=0.,  # set range for random channel shifts
    # set mode for filling points outside the input boundaries
    fill_mode='nearest',
    cval=0.,  # value used for fill_mode = "constant"
    horizontal_flip=True,  # randomly flip images
    vertical_flip=False,  # randomly flip images
    # set rescaling factor (applied before any other transformation)
    rescale=None,
    # set function that will be applied on each input
    preprocessing_function=None,
    # image data format, either "channels_first" or "channels_last"
    data_format=None,
    # fraction of images reserved for validation (strictly between 0 and 1)
    validation_split=0.0)
    
# Compute quantities required for feature-wise normalization
# (std, mean, and principal components if ZCA whitening is applied).
datagen.fit(x_train)

Using real-time data augmentation.


In [0]:
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.1))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.1))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.1))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

**Binary Accuracy**

In [14]:
import keras.backend as K

def binary_accuracy(y_true, y_pred):
    return K.mean(K.equal(y_true, K.round(y_pred)), axis=-1)

model.compile(loss='categorical_crossentropy',
optimizer=keras.optimizers.rmsprop(lr=0.0002, decay=1e-6),
metrics=['accuracy',binary_accuracy])

model.fit_generator(datagen.flow(x_train, y_train,
    batch_size=batch_size),
    epochs=5,
    steps_per_epoch=1563,
    validation_data=(x_test, y_test),
    workers=4)

Instructions for updating:
Use tf.cast instead.
Epoch 1/5
1563/1563 [==============================] - 50s 32ms/step - loss: 1.6657 - acc: 0.3923 - binary_accuracy: 0.9064 - val_loss: 1.3429 - val_acc: 0.5116 - val_binary_accuracy: 0.9167
Epoch 2/5
1563/1563 [==============================] - 45s 29ms/step - loss: 1.3563 - acc: 0.5148 - binary_accuracy: 0.9176 - val_loss: 1.3038 - val_acc: 0.5392 - val_binary_accuracy: 0.9210
Epoch 3/5
1563/1563 [==============================] - 45s 29ms/step - loss: 1.1997 - acc: 0.5735 - binary_accuracy: 0.9257 - val_loss: 1.0254 - val_acc: 0.6409 - val_binary_accuracy: 0.9347
Epoch 4/5
1563/1563 [==============================] - 45s 29ms/step - loss: 1.0995 - acc: 0.6120 - binary_accuracy: 0.9313 - val_loss: 0.9393 - val_acc: 0.6657 - val_binary_accuracy: 0.9396
Epoch 5/5
1563/1563 [==============================] - 45s 29ms/step - loss: 1.0179 - acc: 0.6419 - binary_accuracy: 0.9357 - val_loss: 0.9561 - val_acc: 0.6633 - val_binary_accuracy: 0.93

In [15]:
score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

10000/10000 [==============================] - 1s 144us/step
Test loss: 0.956063318824768
Test accuracy: 0.6633


**Categorical Accuracy**

In [16]:
def categorical_accuracy(y_true, y_pred):
    return K.cast(K.equal(K.argmax(y_true, axis=-1),
                          K.argmax(y_pred, axis=-1)),
                  K.floatx())

model.compile(loss='categorical_crossentropy',
optimizer=keras.optimizers.rmsprop(lr=0.0002, decay=1e-6),
metrics=['accuracy',categorical_accuracy])

model.fit_generator(datagen.flow(x_train, y_train,
    batch_size=batch_size),
    epochs=5,
    steps_per_epoch=1563,
    validation_data=(x_test, y_test),
    workers=4)

Epoch 1/5
1563/1563 [==============================] - 45s 29ms/step - loss: 0.9613 - acc: 0.6617 - categorical_accuracy: 0.6617 - val_loss: 0.9737 - val_acc: 0.6755 - val_categorical_accuracy: 0.6755
Epoch 2/5
1563/1563 [==============================] - 45s 29ms/step - loss: 0.9098 - acc: 0.6814 - categorical_accuracy: 0.6814 - val_loss: 0.8729 - val_acc: 0.7002 - val_categorical_accuracy: 0.7002
Epoch 3/5
1563/1563 [==============================] - 45s 29ms/step - loss: 0.8644 - acc: 0.6986 - categorical_accuracy: 0.6986 - val_loss: 0.8588 - val_acc: 0.7076 - val_categorical_accuracy: 0.7076
Epoch 4/5
1563/1563 [==============================] - 45s 29ms/step - loss: 0.8188 - acc: 0.7159 - categorical_accuracy: 0.7159 - val_loss: 0.8486 - val_acc: 0.7019 - val_categorical_accuracy: 0.7019
Epoch 5/5
1563/1563 [==============================] - 45s 29ms/step - loss: 0.7928 - acc: 0.7244 - categorical_accuracy: 0.7244 - val_loss: 0.7329 - val_acc: 0.7493 - val_categorical_accuracy: 0.

In [17]:
score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

10000/10000 [==============================] - 1s 143us/step
Test loss: 0.7328756074905396
Test accuracy: 0.7493


**Top K Categorical Accuracy**

In [19]:
def top_k_categorical_accuracy(y_true, y_pred, k=5):
    return K.mean(K.in_top_k(y_pred, K.argmax(y_true, axis=-1), k), axis=-1)
  
model.compile(loss='categorical_crossentropy',
optimizer=keras.optimizers.rmsprop(lr=0.0002, decay=1e-6),
metrics=['accuracy',top_k_categorical_accuracy])

model.fit_generator(datagen.flow(x_train, y_train,
    batch_size=batch_size),
    epochs=5,
    steps_per_epoch=1563,
    validation_data=(x_test, y_test),
    workers=4)

Epoch 1/5
1563/1563 [==============================] - 47s 30ms/step - loss: 0.7676 - acc: 0.7321 - top_k_categorical_accuracy: 0.9816 - val_loss: 0.7417 - val_acc: 0.7526 - val_top_k_categorical_accuracy: 0.9791
Epoch 2/5
1563/1563 [==============================] - 46s 30ms/step - loss: 0.7446 - acc: 0.7414 - top_k_categorical_accuracy: 0.9826 - val_loss: 0.6934 - val_acc: 0.7598 - val_top_k_categorical_accuracy: 0.9827
Epoch 3/5
1563/1563 [==============================] - 46s 30ms/step - loss: 0.7219 - acc: 0.7497 - top_k_categorical_accuracy: 0.9843 - val_loss: 0.6936 - val_acc: 0.7649 - val_top_k_categorical_accuracy: 0.9840
Epoch 4/5
1563/1563 [==============================] - 46s 29ms/step - loss: 0.7090 - acc: 0.7550 - top_k_categorical_accuracy: 0.9850 - val_loss: 0.7673 - val_acc: 0.7427 - val_top_k_categorical_accuracy: 0.9824
Epoch 5/5
1563/1563 [==============================] - 46s 29ms/step - loss: 0.6941 - acc: 0.7639 - top_k_categorical_accuracy: 0.9850 - val_loss: 0

In [20]:
score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

10000/10000 [==============================] - 2s 152us/step
Test loss: 0.6981815435409546
Test accuracy: 0.7674


**Custom Metric - IOU**

In [22]:
def iouMetric(y_true, y_pred):
   pred = K.cast(K.greater(y_pred, 0.5), K.floatx())
   union = K.cast(K.greater(y_true + pred, 0), K.floatx())
   intersec = y_true * pred
   iou = K.sum(intersec) / (K.sum(union) + K.epsilon())
 
   return iou

model.compile(loss='categorical_crossentropy',
optimizer=keras.optimizers.rmsprop(lr=0.0002, decay=1e-6),
metrics=['accuracy',iouMetric])

model.fit_generator(datagen.flow(x_train, y_train,
    batch_size=batch_size),
    epochs=5,
    steps_per_epoch=1563,
    validation_data=(x_test, y_test),
    workers=4)

Epoch 1/5
1563/1563 [==============================] - 46s 29ms/step - loss: 0.6842 - acc: 0.7639 - iouMetric: 0.6161 - val_loss: 0.6887 - val_acc: 0.7684 - val_iouMetric: 0.6316
Epoch 2/5
1563/1563 [==============================] - 45s 29ms/step - loss: 0.6737 - acc: 0.7684 - iouMetric: 0.6222 - val_loss: 0.7423 - val_acc: 0.7559 - val_iouMetric: 0.6140
Epoch 3/5
1563/1563 [==============================] - 45s 29ms/step - loss: 0.6727 - acc: 0.7686 - iouMetric: 0.6209 - val_loss: 0.6951 - val_acc: 0.7649 - val_iouMetric: 0.6247
Epoch 4/5
1563/1563 [==============================] - 45s 29ms/step - loss: 0.6602 - acc: 0.7752 - iouMetric: 0.6303 - val_loss: 0.7155 - val_acc: 0.7697 - val_iouMetric: 0.6227
Epoch 5/5
1563/1563 [==============================] - 45s 29ms/step - loss: 0.6622 - acc: 0.7738 - iouMetric: 0.6286 - val_loss: 0.6692 - val_acc: 0.7886 - val_iouMetric: 0.6576


In [23]:
score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

10000/10000 [==============================] - 1s 147us/step
Test loss: 0.6691593486785888
Test accuracy: 0.7886


**The training efficiency on GPU is much higher than on CPU.**